In [1]:
import tweepy
import tqdm
import csv
import json
import time
from tqdm import tqdm_notebook as tqdm

In [2]:
def makeAuthConnection():
    consumerApiKey = 'Tx4p4obJsXFwO5eHKEbsfidJq'
    consumerApiSecret = 'uoxlO5zt48FYPjEJjWEa49ukHe1oWsWahvPCwbf6ncudRi713j'
    acessToken = '294569456-ZwBlFWdoYP9OazssEsxnoargsaAPrWHWfjUj6cH3'
    acessTokenSecret  = 'J6fCig6s1tsnDtn26ISbZRPVQryRWtfHftb76MNnQLj9u'
    
    auth = tweepy.OAuthHandler(consumerApiKey, consumerApiSecret)
    #auth = tweepy.AppAuthHandler(consumerApiKey, consumerApiSecret)
    auth.set_access_token(acessToken, acessTokenSecret)
    
    return tweepy.API(auth , wait_on_rate_limit = True,wait_on_rate_limit_notify = True)

In [3]:
api = makeAuthConnection()
# for status in tweepy.Cursor(api.search, q='tweepy').items(10):
#     print(status.text)

In [4]:
def checkRemainingSearchCount():
    jsonString = api.rate_limit_status()['resources']['search']['/search/tweets']
    upperLimit = jsonString['limit']
    remiaingFetch = jsonString['remaining']
    #resetTime = jsonString['reset']/60000 
    print (jsonString)
    return upperLimit,remiaingFetch

In [5]:
checkRemainingSearchCount()

{'limit': 180, 'remaining': 180, 'reset': 1573904010}


(180, 180)

In [6]:
# searchString = '##housefull4review'
# searchFilter = ' AND -filter:retweets AND -filter:replies AND -filter:links and -filter:videos'
# tweetList = tweepy.Cursor(api.search,q = searchString + searchFilter ,
#                             lang  = 'en',include_entities=False,tweet_mode='extended',count = 100)
# print(type(tweetList))
# print(tweetList)
# for tweets in tweetList.items(50):
#     jsonString = tweets._json
#     print(jsonString['id'], jsonString['created_at'],jsonString['full_text'].replace('\n', ' '))
#     print(tweets.full_text)

In [7]:
# This method will generate a file containng the tweets of the data 
# This uses the tweepy API to fetch the data
# TODO This method generate the maxind tweets twice. Will have to check on it.
def searchTweetsByHashtag(searchlist):
    # use this filter to filter the tweets based on the key words -filter:retweets AND -filter:replies
    searchFilter = ' AND -filter:links and -filter:videos and -filter:retweets'
    fileName = 'tweetDataset.csv'
    with open (fileName,'a', newline='',encoding='utf-8') as sampleFile:
        writer = csv.writer(sampleFile,quoting = csv.QUOTE_NONNUMERIC)
        try:
            for searchString in searchlist: 
                search_result = api.search(q=searchString + searchFilter,count=1,lang="en",tweet_mode='extended'
                                           , result_type  = 'recent')
                if(len(search_result) == 0):
                    print("*************No data on "+ searchString +" hashtag.***************")
                else : 
                    max_id = search_result[0].id
                    #print("max_id",max_id)
                    old_id = -1
                    i = 1
                    while(max_id != old_id):
                        old_id = max_id
                        tweetDic = tweepy.Cursor(api.search,q = searchString + searchFilter  ,lang  = 'en'
                                                 ,include_entities=False,tweet_mode='extended',count = 100
                                                 ,max_id = max_id).items(300)
                        print("loop count",i)
                        for tweets in tweetDic:
                            jsonString = tweets._json
                            #print(jsonString['id'],jsonString['full_text'].replace('\n', ' '))
                            csv_row = [jsonString['id'],jsonString['user']['screen_name'],jsonString['retweet_count']
                                       ,jsonString['full_text'].replace('\n', ' ')]  
                            # we can also encode the text here to remove emojies from the text.
                            max_id = jsonString['id'] + 1
                            writer.writerow(csv_row)
                        print("Going to sleep to keep limit to check")    
                        time.sleep(3)
                        print("Waking Up")
                print("*************No more data to exact.*************")
        except tweepy.TweepError as e:
            print("Some error!!:"+str(e))

In [8]:
search_criteria = ['#MotichoorChaknachoorReview','#jhalkireview','#FordVsFerrari','#MotherlessBrooklyn'
                   ,'#Charlie\'sAngels','#DoctorSleepReview','#MidwayMovie','#Actionreview','#SangathamizhanReview'
                  ,'#JhalleReview']
searchTweetsByHashtag(search_criteria)

loop count 1
Going to sleep to keep limit to check
Waking Up
loop count 1
Going to sleep to keep limit to check
Waking Up
*************No more data to exact.*************
loop count 1
Going to sleep to keep limit to check
Waking Up
loop count 1
Going to sleep to keep limit to check
Waking Up
*************No more data to exact.*************
loop count 1
Going to sleep to keep limit to check
Waking Up
loop count 1
Going to sleep to keep limit to check
Waking Up
*************No more data to exact.*************
loop count 1
Going to sleep to keep limit to check
Waking Up
loop count 1
Going to sleep to keep limit to check
Waking Up
*************No more data to exact.*************
*************No data on #Charlie'sAngels hashtag.***************
*************No more data to exact.*************
*************No data on #DoctorSleepReview hashtag.***************
*************No more data to exact.*************
loop count 1
Going to sleep to keep limit to check
Waking Up
loop count 1
Going to sle